In [ ]:
# GET POR Files Already Downloaded
import glob
import shutil

source = '/Users/kris/Downloads/FFIEC/'
destination = '/Users/kris/Library/CloudStorage/OneDrive-Personal/Documents/Visual Studio 2019/Repos/ffiec-xbrl/PubData/Src/RSSDs/'

for src in glob.glob(source + '*/* POR *'):
    shutil.copy(src, destination)

In [1]:
# Imports and Constants
import pandas as pd
import numpy as np

#dtypes = {'RSSD_ID':str,
#          'FDIC_Certificate_Number':str,
#          'OCC_Charter_Number':str,
#          'OTS_Docket_Number':str,
#          'Primary_ABA_Routing_Number':str,
#          'Financial_Institution_Name': str,
#          'Financial_Institution_Address': str,
#          'Financial_Institution_City': str,
#          'Financial_Institution_State': str,
#          'Financial_Institution_Zip_Code': str,
#          'Financial_Institution_Filing_Type': str,
#          'Last Date/Time Submission Updated On': 'str'}

dtypes = {'IDRSSD': np.dtype('str'),
          'FDIC Certificate Number': np.dtype('str'),
          'OCC Charter Number': np.dtype('str'),
          'OTS Docket Number': np.dtype('str'),
          'Primary ABA Routing Number': np.dtype('str'),
          'Financial Institution Name': np.dtype('str'),
          'Financial Institution Address': np.dtype('str'),
          'Financial Institution City': np.dtype('str'),
          'Financial Institution State': np.dtype('str'),
          'Financial Institution Zip Code': np.dtype('str'),
          'Financial Institution Filing Type': np.dtype('str'),
          'Last Date/Time Submission Updated On': np.dtype('str')}

filepath_in = 'PubData/Src/RSSDs/'
filepath_out = 'PubData/'
filename = 'FFIEC CDR Call Bulk POR 03312024.txt'

In [ ]:
# Compare Bank_Dim and RSSD_Dict
df_bank = pd.read_csv(filepath_out + 'Bank_Dim.csv', sep=',', index_col=False, quotechar='"')
df_rssd = pd.read_csv(filepath_out + 'RSSD_Dict.csv', sep=',', index_col=False, quotechar='"')

print(df_bank.shape, df_rssd.shape)


In [ ]:
#Open File (Bank_dim from RSSD_Por)

#with open(filename, 'r') as file:
df = pd.read_csv(filepath_in + filename, sep='\t', index_col=False, quotechar='"', dtype = dtypes, parse_dates=['Last Date/Time Submission Updated On'])

In [ ]:
#Clean RSSD_Por File
df = df.apply(lambda x: x.str.strip() if x.dtype.name == 'object' else x, axis=0)

In [ ]:
#Clean RSSD_Por Format - Align with and Create RSSD_Dict.csv
df.rename(columns={'IDRSSD': 'RSSD_ID'}, inplace=True)
df.columns = df.columns.str.replace(' ', '_')

#Create RSSD_Dict file
#df.iloc[:25].to_csv(filepath_out + 'RSSD_Dict.csv', sep=',', quotechar='"', index= False,)
df.to_csv(filepath_out + 'RSSD_Dict.csv', sep=',', quotechar='"', index= False)

In [ ]:
#Confirm data
df[:5]

#df.Financial_Institution_Address
df.iloc[0,6]

In [ ]:
from FDIC import ETL, ExEngine

# ExEngine.FillMaster('504713')
#ETL.GenBankDim()
ETL.DownloadCallReports(['XBRL'])

#ETL.ParseXBRL()
#ETL.GenBankMaster()
#ETL.GenCallMaster()

In [ ]:
from FDIC import ETL
call_report_path = '/Users/kris/data_sources/fdic/BulkReports/ACNB BANK_5210/ACNB BANK_5210_20010331.XBRL'
bank_path = '/Users/kris/data_sources/fdic/BulkReports/ACNB BANK_5210/'
folder_path = '/Users/kris/data_sources/fdic/BulkReports/'

#tot = ETL.ParseXBRL(call_report_path)
ETL.GenBankMaster()
#ETL.GenCallMaster()

In [ ]:
ETL.GenCallMaster()

In [2]:
import FDIC.constants as paths

#tot

df = pd.read_csv(paths.localPath + paths.filename_MasterCall)


In [3]:
df.head(1)

,ReportPeriodEndDate,RSSD_ID,Item_Name,MDRM_Item,Confidential,Value
0,2006-06-30,28013,CASH AND BALANCES DUE FROM DEPOSITORY INSTITUT...,RCON0010,No,40009000


In [ ]:
#Display methods of data
g_df = df[df['RSSD_ID']==28013].groupby(['MDRM_Item', 'RSSD_ID', 'ReportPeriodEndDate'])['MDRM_Item'].count()

g_df = df[df['RSSD_ID']==28013].groupby(['MDRM_Item', 'RSSD_ID', 'ReportPeriodEndDate'])['MDRM_Item'].count()

In [11]:
#Number of reporters by field by date
pvt = df[['MDRM_Item', 'ReportPeriodEndDate']].pivot_table(
    index=['MDRM_Item'],
    columns=['ReportPeriodEndDate'],
    values=['MDRM_Item'],
    aggfunc={'MDRM_Item': ['count']}
)
pvt.fillna(0, inplace=True)
pvt

                     MDRM_Item                                              \
                         count                                               
ReportPeriodEndDate 2001-03-31 2001-06-30 2001-09-30 2001-12-31 2002-03-31   
MDRM_Item                                                                    
RCFA3792                   0.0        0.0        0.0        0.0        0.0   
RCFA5310                   0.0        0.0        0.0        0.0        0.0   
RCFA5311                   0.0        0.0        0.0        0.0        0.0   
RCFA7204                   0.0        0.0        0.0        0.0        0.0   
RCFA7205                   0.0        0.0        0.0        0.0        0.0   
...                        ...        ...        ...        ...        ...   
TEXTG561                   0.0        0.0        0.0        0.0        0.0   
TEXTG571                   0.0        0.0        0.0        0.0        0.0   
TEXTG576                   0.0        0.0        0.0        0.0 

In [32]:
#Number of RSSD's by date
pvt = df[['ReportPeriodEndDate', 'RSSD_ID']].drop_duplicates().reset_index().pivot_table(
    index=['ReportPeriodEndDate'],
    values=['RSSD_ID'],
    aggfunc={'RSSD_ID': ['count']}
).sort_values(by='ReportPeriodEndDate', ascending=False)

pvt = pvt.set_axis([col for col in pvt.keys()], axis=1)
#pvt.reset_index(inplace=True)

pvt.fillna(0, inplace=True)
pvt

,"(RSSD_ID, count)"
ReportPeriodEndDate,
2024-03-31,161
2023-12-31,161
2023-09-30,161
2023-06-30,161
2023-03-31,161
...,...
2002-03-31,146
2001-12-31,146
2001-09-30,146


In [28]:
tdf = df[df['ReportPeriodEndDate']=='2001-03-31'][['ReportPeriodEndDate', 'RSSD_ID']].drop_duplicates().reset_index()
tdf


,index,ReportPeriodEndDate,RSSD_ID
0,71759,2001-03-31,28013
1,137783,2001-03-31,14865
2,266146,2001-03-31,45430
3,313390,2001-03-31,42448
4,498013,2001-03-31,44433
...,...,...,...
141,16649751,2001-03-31,35057
142,16840347,2001-03-31,12946
143,16945983,2001-03-31,5461
144,17003757,2001-03-31,7456


In [ ]:
#df
grouped_df = df[['MDRM_Item', 'Item_Name']].drop_duplicates().groupby(['MDRM_Item', 'Item_Name']).size().reset_index(name='Count')
print(grouped_df)